In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt

## File paths

In [2]:
train_dataset_path = "sign-language-mnist/sign_mnist_train/sign_mnist_train.csv"
test_dataset_path =  "sign-language-mnist/sign_mnist_test/sign_mnist_test.csv"
train_dataset = pd.read_csv(train_dataset_path)
test_dataset = pd.read_csv(test_dataset_path)


## Load train and test files

In [3]:
def load_dataset():
    train_set_x_orig = np.array(train_dataset.iloc[:,1:]) # train set features
    train_set_y_orig = np.array(train_dataset.iloc[:,:1]) # train set labels
    test_set_x_orig = np.array(test_dataset.iloc[:,1:]) # test set features
    test_set_y_orig = np.array(test_dataset.iloc[:,:1]) # test set labels
    
    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
    
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig

In [4]:
# Function to convert to one hot encoding
def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)].T
    return Y

In [5]:
# Flatten the data
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig = load_dataset()
X_train_flatten = X_train_orig.reshape(X_train_orig.shape[0], -1).T
X_test_flatten = X_test_orig.reshape(X_test_orig.shape[0], -1).T
X_train = X_train_flatten/255.
X_test = X_test_flatten/255.
Y_train = convert_to_one_hot(Y_train_orig, 25)
Y_test = convert_to_one_hot(Y_test_orig, 25)

In [6]:
print ("Number of training examples = " + str(X_train.shape[1]))
print ("Number of test examples = " + str(X_test.shape[1]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

Number of training examples = 27455
Number of test examples = 7172
X_train shape: (784, 27455)
Y_train shape: (25, 27455)
X_test shape: (784, 7172)
Y_test shape: (25, 7172)


In [11]:
# Create Tensors
def create_placeholders(n_x,n_y):
    """
    Creates the placeholders for the tensorflow session.
    
    Arguments:
    n_x -- size of an image vector (num_px * num_px = 28 * 28 * 1 = 784), 1 because it is a grayscale image
    n_y -- number of classes (from 0 to 24)
    
    Returns:
    X -- placeholder for the data input, of shape [n_x, None] and dtype "tf.float32"
    Y -- placeholder for the input labels, of shape [n_y, None] and dtype "tf.float32"
    """
    X = tf.placeholder(shape=[n_x,None],dtype=tf.float32,name="X")
    Y = tf.placeholder(shape=[n_y,None],dtype=tf.float32,name="Y")
    
    return X, Y
        

In [15]:
# Initialization of parameters ie. Weights, Biases
def initialize_parameters():
    """
    Initializes the parameters for the deep neural network with 4 hidden layers
    
    Returns:
    Parameters : A dictionary with the weights and the biases
    """
    W1 = tf.get_variable("W1",[35,784], initializer=tf.contrib.layers.xavier_initialization())
    b1 = tf.get_variable("b1",[35,1], initializer=tf.contrib.layers.xavier_initialization())
    W2 = tf.get_variable("W2",[30,35], initializer=tf.contrib.layers.xavier_initialization())
    b2 = tf.get_variable("b2",[30,1], initializer=tf.contrib.layers.xavier_initialization())
    W3 = tf.get_variable("W3",[25,30], initializer=tf.contrib.layers.xavier_initialization())
    b3 = tf.get_variable("b3",[25,1], initializer=tf.contrib.layers.xavier_initialization())
    parameters = {"W1":W1,
                  "b1":b1,
                  "W2":W2,
                  "b2":b2,
                  "W3":W3,
                  "b3":b3
                 }
    
    return parameters
 


In [16]:
# Forward Propagation
def forward_prop(X, parameters):
    """
    Function for forward propagation of the computation graph of the deep neural network
    Args : 
    X : Input dataset , images (grayscale)
    parameters : Dictionary that contains the weights and the biases
    
    Returns : 
    Z3 : Output of the final linear unit
    """
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    W3 = parameters['W3']
    b3 = parameters['b3']
    
    # Calculate the Linear function and activation values for each layer
    Z1 = tf.matmul(W1,X)+b1
    A1 = tf.nn.relu(Z1)
    Z2 = tf.matmul(W2,A1)+b2
    A2 = tf.nn.relu(Z2)
    Z3 = tf.matmul(W3,A2)+b3
    
    return Z3
    